In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from datautility import *
from dataset import *
from vnet import *
from training import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, sampler, SubsetRandomSampler
from torchvision import transforms, utils

import torch.nn.functional as F  # useful stateless functions
import torchvision.transforms as T

#------------------------------- GLOBAL VARIABLES -------------------------------------#

USE_GPU = True
BATCH_SIZE = 2
NUM_WORKERS = 8
NUM_TRAIN = 72 # 80 training sample and 37 validation sample
LEARNING_RATE = 1e-3

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
    print('using GPU for training')
else:
    device = torch.device('cpu')
    print('using CPU for training')

using GPU for training


## If Training new data only
* Positive: 67 samples
* Negative: 50 samples

In [3]:
###################### Importing Packages #########################################
# metric = ['ad', 'ak', 'awf', 'eas_De_par', 'eas_De_perp', 'eas_tort', 'FA', 'ias_Da', 'md', 'mk', 'rd', 'rk']
# 12 metrics in total

metric = ['ak', 'awf', 'eas_De_par', 'eas_De_perp', 'fa', 'ias_Da', 'md', 'mk']
# 8 metric to use, proposed by Alp
    
NEW_PATH = './data/117_Stats_Rep.xls'

new_data = pd.read_excel(NEW_PATH, header=0, index_col=0, sheet_name=0)
new_idx_ori = list(new_data.index)

new_idx = [i[3:-4] for i in new_idx_ori]

print(new_idx, len(new_idx))

['TBI001', 'TBI002', 'TBI003', 'TBI004', 'TBI005', 'TBI006', 'TBI007', 'TBI008', 'TBI010', 'TBI011', 'TBI012', 'TBI013', 'TBI014', 'TBI015', 'TBI019', 'TBI020', 'TBI021', 'TBI022', 'TBI023', 'TBI024', 'TBI025', 'TBI026', 'TBI027', 'TBI028', 'TBI029', 'TBI031', 'TBI032', 'TBI033', 'TBI035', 'TBI036', 'TBI037', 'TBI038', 'TBI039', 'TBI040', 'TBI041', 'TBI042', 'TBI043', 'TBI044', 'TBI045', 'TBI046', 'TBI047', 'TBI048', 'TBI049', 'TBI050', 'TBI051', 'TBI052', 'TBI053', 'TBI054', 'TBI055', 'TBI056', 'TBI057', 'TBI058', 'TBI059', 'TBI060', 'TBI061', 'TBI062', 'TBI063', 'TBI064', 'TBI065', 'TBI066', 'TBI067', 'TBI068', 'TBI069', 'TBI070', 'TBI071', 'TBI072', 'TBI073', 'TBN001', 'TBN002', 'TBN003', 'TBN004', 'TBN005', 'TBN006', 'TBN007', 'TBN008', 'TBN009', 'TBN010', 'TBN011', 'TBN012', 'TBN013_2', 'TBN014_2', 'TBN018', 'TBN019', 'TBN020', 'TBN021', 'TBN022', 'TBN023', 'TBN024', 'TBN025', 'TBN026', 'TBN027', 'TBN028', 'TBN029', 'TBN030', 'TBN031', 'TBN032', 'TBN033', 'TBN034', 'TBN035', 'TBN0

In [4]:
regen = False

if regen:
    data_index = np.arange(117)
    data_index = np.random.shuffle(data_index)
    print(list(data_index))
    
else:
    data_index = np.array ([27, 22, 83, 13, 57, 33, 53, 110, 58, 72, 12, 66, 69, 39, 105, 60, 42, 114, 91, 6, 80, 101,
                            73, 87, 26, 74, 28, 19, 25, 79, 70, 1, 18, 2, 38, 99, 40, 48, 35, 20, 78, 102, 14, 54, 84,
                            34, 77, 47, 9, 16, 59, 11, 112, 62, 71, 44, 85, 31, 51, 37, 94, 75, 21, 88, 68, 30, 104, 15,
                            55, 61, 41, 115, 29, 49, 52, 107, 82, 45, 76, 89, 86, 92, 81, 95, 8, 116, 65, 106, 32, 56, 46,
                            100, 36, 103, 90, 5, 43, 50, 67, 64, 111, 10, 63, 97, 17, 93, 98, 24, 7, 3, 0, 113, 108, 96, 4, 109, 23]
                            )

dataset_image = MTBIDatasetSub(new_index = new_idx,
                            shuffle_index = data_index, 
                            metric = metric, 
                            transform=transforms.Compose([
                                 RandomAffine(180, 15)
                            ]),
                     )

#-------------------------CREATE DATA LOADER FOR TRAIN AND VAL------------------------#

data_size = len(dataset_image)

train_loader = DataLoader(dataset_image, batch_size=BATCH_SIZE, \
                    sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)),\
                    num_workers=NUM_WORKERS)
validation_loader = DataLoader(dataset_image, batch_size=BATCH_SIZE,
                    sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN,data_size)),\
                    num_workers=NUM_WORKERS)

# data_set = MTBIDataset(image_dict, metric, transform=None, mode='new')

In [5]:
for i_batch, sample_batched in enumerate(train_loader):
    print(i_batch, sample_batched['image'].size(), sample_batched['label'].size())
    # observe 4th batch and stop.
    if i_batch == 1:
        # show_batch_image(sample_batched['image'],sample_batched['label'],BATCH_SIZE)
        break

0 torch.Size([2, 8, 96, 96, 96]) torch.Size([2, 1])
1 torch.Size([2, 8, 96, 96, 96]) torch.Size([2, 1])


In [6]:
#-------------------------NEW MODEL INIT WEIGHT--------------------------------------#

LoadCKP = False
CKPPath = 'checkpoint2019-03-31 13:33:50.772063.pth'

model = LNet(img_size=(96, 96, 96))
model.apply(weights_init)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=50, verbose=True)

if LoadCKP:
    model, optimizer, scheduler = loadckp(model, optimizer, scheduler, CKPPath, device=device)

In [ ]:
loss = nn.CrossEntropyLoss()

train(model, train_loader, validation_loader, optimizer, scheduler, device, dtype, lossFun=loss, epochs=5000, streopch=0)

Epoch 0 finished ! Training Loss: 0.9499, acc: 0.5556
     validation loss = 3.5069, accuracy = 0.4783
Checkpoint 1 saved !
Epoch 1 finished ! Training Loss: 0.7202, acc: 0.6111
     validation loss = 0.7215, accuracy = 0.5000
Epoch 2 finished ! Training Loss: 0.8311, acc: 0.4444
     validation loss = 0.7758, accuracy = 0.5000
Epoch 3 finished ! Training Loss: 0.7445, acc: 0.5833
     validation loss = 0.7297, accuracy = 0.5217
Epoch 4 finished ! Training Loss: 0.7271, acc: 0.5556
